In [1]:
import QUANTAXIS as QA
import datetime
import talib
import traceback
from finance_tools_py.backtest import BackTest
from finance_tools_py.backtest import MinAmountChecker
import finance_tools_py
from finance_tools_py.simulation import Simulation
from finance_tools_py.simulation.callbacks import CallBack
from finance_tools_py.simulation.callbacks.talib import DEMA
from finance_tools_py.simulation.callbacks import Rolling_Future
import pandas as pd
import numpy as np
import concurrent.futures
import os

In [2]:
filename="DEMA双线交叉.csv"

start='2005-01-01'

code_list=QA.QA_fetch_get_stock_list('pytdx')
## 合并所有股票一同回测
from tqdm.auto import tqdm
qfqs={}
hfqs={}
for symbol in tqdm(code_list['code'].values,"Reading datas..."):
    try:
        data=QA.QA_fetch_stock_day_adv(symbol,start=start,end='2020-03-17')
        if data and not data.data.empty:
            qfq=data.to_qfq().data.reset_index()
            qfq['date']=pd.to_datetime(qfq['date'])
            qfqs[symbol]=qfq
            hfq=data.to_hfq().data.reset_index()
            hfq['date']=pd.to_datetime(hfq['date'])
            hfqs[symbol]=hfq
    except:
        traceback.print_exc()
        continue
print('Datas readed.')
code_list=qfqs.keys()

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
USING DEFAULT FUTURE IP


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 124.160.88.183 future_ip 124.74.236.94


QA Error QA_fetch_stock_day_adv parameter code=002977 , start=2005-01-01, end=2020-03-17 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=002978 , start=2005-01-01, end=2020-03-17 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300819 , start=2005-01-01, end=2020-03-17 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300821 , start=2005-01-01, end=2020-03-17 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300822 , start=2005-01-01, end=2020-03-17 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300823 , start=2005-01-01, end=2020-03-17 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300825 , start=2005-01-01, end=2020-03-17 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=603221 , start=2005-01-01, end=2020-03-17 call QA_fetch_stock_day return None


In [3]:
class chker(MinAmountChecker):
    def __init__(self, buy_dict, sell_dict, min_price=3000, min_increase=1.15, **kwargs):
        super().__init__(buy_dict, sell_dict, **kwargs)
        self._min_price = min_price
        self._min_increase = min_increase

    def on_check_sell(self,
                      date: datetime.datetime.timestamp,
                      code: str,
                      price: float,
                      cash: float,
                      hold_amount: float,
                      hold_price: float) -> bool:
        if price < hold_price:
            # 当前价格小于持仓价时，不可卖
            return False
        if code in self.sell_dict.keys() and date in self.sell_dict[code]:
            # 其他情况均可卖
            return True
        if hold_amount > 0 and self._min_increase > 0 and price >= hold_price * self._min_increase:
            # 当前价格超过成本价15%时，可卖
            return True
        return False

    def on_calc_buy_amount(self, date, code: str, price: float, cash: float) -> float:
        amount = 100
        if self._min_price > 0:
            if cash < self._min_price:
                return super().on_calc_buy_amount(date, code, price, cash)
            while True:
                amount = amount + 100
                if price * amount > self._min_price:
                    break
            amount = amount - 100
        return amount

    def on_calc_sell_amount(self,
                            date: datetime.datetime.timestamp,
                            code: str,
                            price: float,
                            cash: float,
                            hold_amount: float,
                            hold_price: float) -> float:
        """返回所有持仓数量，一次卖出所有"""
        return hold_amount


class Cross_DEMA(CallBack):
    def __init__(self,t1,t2):
        self.t1=t1
        self.t2=t2

    """双线相交法"""
    def on_preparing_data(self, data, **kwargs):
        data['{}日线上穿{}日线'.format(self.t1,self.t2)]=0
        data['{}日线下穿{}日线'.format(self.t1,self.t2)]=0
        #下穿。t1.dema<t2.dema & t1.dema.shift>t2.dema.shift
        data.loc[(data['dema_close_{}'.format(self.t1)]<data['dema_close_{}'.format(self.t2)]) & (data['dema_close_{}'.format(self.t1)].shift()>data['dema_close_{}'.format(self.t2)].shift()),'{}日线下穿{}日线'.format(self.t1,self.t2)]=1
        #上穿。t1.dema>t2.dema & t1.dema.shift<t2.dema.shift
        data.loc[(data['dema_close_{}'.format(self.t1)]>data['dema_close_{}'.format(self.t2)]) & (data['dema_close_{}'.format(self.t1)].shift()<data['dema_close_{}'.format(self.t2)].shift()),'{}日线上穿{}日线'.format(self.t1,self.t2)]=1

class Rolling_Future_New(Rolling_Future):
    def on_preparing_data(self, data, **kwargs):
        super().on_preparing_data(data, **kwargs)
        data['未来{}日涨跌'.format(self.timeperiod)]=np.sign(data['rolling_{}_{}_mean'.format(self.col_name,self.timeperiod)]-data[self.col_name])
        data.rename(columns={'rolling_close_{}_mean/close'.format(self.timeperiod):'未来 {} 日均价/当前'.format(self.timeperiod)},inplace=True)

In [4]:
def test(datas,t1=5,t2=20):
    """
    datas: 前复权数据字典
    """
    simdatas={}
    buys={}
    sells={}
    for k,v in tqdm(datas.items(),"分析买卖点..."):

        s=Simulation(v,'',callbacks=[DEMA(t1),
                        DEMA(t2),
                        Cross_DEMA(t1,t2),
                        Rolling_Future_New(timeperiod=t1),
                        Rolling_Future_New(timeperiod=t2)])
        s.simulate()
        simdatas[k]=s.data
        buys[k]=s.data[(s.data['date']>start) & (s.data['date']<'2019-01-01')].loc[s.data['{}日线下穿{}日线'.format(t1,t2)]==1]['date'].dt.to_pydatetime()
        sells[k]=s.data[(s.data['date']>start) & (s.data['date']<'2019-01-01')].loc[s.data['{}日线下穿{}日线'.format(t1,t2)]==1]['date'].dt.to_pydatetime()
    
    return simdatas,buys,sells

In [5]:
alldata_fqf=pd.concat(hfqs.values()).sort_values('date')

In [6]:
result={}
for t1 in range(3,31):
    for t2 in range(3,6):
        t2=t1*t2
        
        simdatas,buys,sells=test(qfqs,t1=t1,t2=t2)
        
        bt = BackTest(alldata_fqf, init_cash=50000, callbacks=[chker(
             buy_dict=buys,
             sell_dict=sells)])
        bt.calc_trade_history(verbose=0)
        print(bt.report(show_history=False))
        cur=0
        if not bt.hold_price_cur_df.empty:
            cur=np.round(bt.available_cash + sum(bt.hold_price_cur_df['amount']*bt.hold_price_cur_df['price_cur']),2)
        df=pd.DataFrame(
            {
                'available_cash':[np.round(bt.available_cash,2)],
                'total_assets_cur':[cur],
                'total_assets_buy':[np.round(bt.total_assets_cur,2)],
                'buy_cur_diff':[np.round(cur-bt.total_assets_cur,2)],
                'trade_times':[len(bt.history_df)],
                'init_cash':[bt.init_cash],
                't1':[t1],
                't2':[t2]
            })
        print(df)
        if os.path.exists(filename):
            df=pd.read_csv(filename,index_col=False).append(df)
        df.to_csv(filename,index=False)

C:\Users\guqiang\AppData\Local\conda\conda\envs\stock_ai\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in sign



数据时间:2005-01-04 00:00:00~2020-03-09 00:00:00（可交易天数3688）
初始资金:50000.00
交易次数:16883 (买入/卖出各算1次)
可用资金:6384.75
当前持仓:        buy_price  amount  price_cur
code                                
000521   7.755975   100.0   7.293191
000537  21.325662   100.0  12.209716
000571  12.848637   200.0   3.294242
600026  11.325617   100.0  10.512334
600601  14.654043   200.0   6.166599
601518   3.898520   100.0   3.007138
当前总资产:16315.87
资金变化率:12.77%
资产变化率:32.63%
总手续费:84449.55
总印花税:22243.99
   available_cash  total_assets_cur  total_assets_buy  buy_cur_diff  \
0         6384.75          11579.16          16315.87      -4736.71   

   trade_times  init_cash  t1  t2  
0        16883      50000   3   9  


C:\Users\guqiang\AppData\Local\conda\conda\envs\stock_ai\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in sign



数据时间:2005-01-04 00:00:00~2020-03-09 00:00:00（可交易天数3688）
初始资金:50000.00
交易次数:18032 (买入/卖出各算1次)
可用资金:3333.05
当前持仓:        buy_price  amount  price_cur
code                                
000897   7.371168   100.0   6.294212
000960  60.752245   100.0  20.170619
002145  30.190000   100.0  13.229904
600228  29.362500   100.0   8.275500
600959  53.280000   100.0   5.097635
当前总资产:21428.64
资金变化率:6.67%
资产变化率:42.86%
总手续费:90359.94
总印花税:24826.66
   available_cash  total_assets_cur  total_assets_buy  buy_cur_diff  \
0         3333.05           8639.84          21428.64      -12788.8   

   trade_times  init_cash  t1  t2  
0        18032      50000   3  12  


C:\Users\guqiang\AppData\Local\conda\conda\envs\stock_ai\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in sign


...


数据时间:2005-01-04 00:00:00~2020-03-09 00:00:00（可交易天数3688）
初始资金:50000.00
交易次数:17610 (买入/卖出各算1次)
可用资金:19400.82
当前持仓:         buy_price  amount  price_cur
code                                 
000150   42.680000   100.0  10.036874
000681   36.733667   300.0  22.108295
000683   14.581713   500.0   7.768153
000686  120.155002   200.0  54.608642
000766   15.680000   100.0   6.280000
000831   89.944511   100.0  26.047500
000882    7.270701   100.0   4.449920
000913   27.769055   100.0  15.732442
002164   30.521796   100.0  11.988650
002260  106.251557   100.0   7.275922
002406   30.938543   100.0  21.194819
002576   25.605368   100.0  13.182553
002603   47.129119   200.0  42.981690
300189   48.235459   100.0  21.783403
300233   63.503579   100.0  43.728737
300490   37.446709   200.0  21.395354
600027    7.619221   100.0   5.818673
600033   21.560910   100.0  13.643438
600035   15.088042   100.0   9.222024
600135   22.086836   100.0  11.174984
600180   39.196176   200.0   9.561255
600208   37.4

C:\Users\guqiang\AppData\Local\conda\conda\envs\stock_ai\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in sign



数据时间:2005-01-04 00:00:00~2020-03-09 00:00:00（可交易天数3688）
初始资金:50000.00
交易次数:10159 (买入/卖出各算1次)
可用资金:17.67
当前持仓:         buy_price  amount  price_cur
code                                 
000831   85.827247   200.0  26.047500
002087   15.520002   100.0  15.443066
002102   52.079902   100.0  19.726783
002162   11.373449   300.0   7.984206
002438   60.397373   100.0  32.905316
300288  458.265570   100.0  78.674044
600120   41.568299   100.0  38.712988
600272   14.001812   100.0   8.759484
600339   12.827959   200.0   5.293691
600601    9.829221   200.0   6.166599
600657   12.730588   100.0   9.032633
600828   43.923138   100.0  16.476527
601106    4.135493   300.0   2.801788
601519   83.294239   100.0  24.168126
601600    5.397604   100.0   3.163057
601886   22.241519   100.0  15.126443
601918    6.803350   100.0   4.459477
603011   26.009471   100.0  15.419909
当前总资产:110590.52
资金变化率:0.04%
资产变化率:221.18%
总手续费:52690.36
总印花税:25397.55
   available_cash  total_assets_cur  total_assets_buy  buy_c

C:\Users\guqiang\AppData\Local\conda\conda\envs\stock_ai\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in sign



数据时间:2005-01-04 00:00:00~2020-03-09 00:00:00（可交易天数3688）
初始资金:50000.00
交易次数:9332 (买入/卖出各算1次)
可用资金:21096.14
当前持仓:         buy_price  amount  price_cur
code                                 
000035   26.784000   100.0  18.648000
000156   58.620591   100.0  17.201370
000498   11.215800   100.0   7.451496
000541   57.478364   100.0  36.279570
000551   21.870396   100.0  18.590446
000633   11.649000   100.0   4.158000
000665   22.588155   100.0  14.081345
000686  117.781695   100.0  54.608642
002583  104.996019   100.0  43.872712
002616   49.086586   100.0  36.377509
300165   86.410327   100.0  33.224770
300230   55.907679   100.0  37.652902
600103    7.407406   100.0   4.891588
600119   25.661685   200.0   8.448106
600139   53.016036   100.0  19.561734
600202   44.226056   100.0  10.529265
600279   14.930876   100.0   9.270909
600819   10.390961   100.0   7.622381
601555   25.914236   100.0  10.953864
601633   51.158359   100.0  31.454093
601677   12.220475   100.0  12.163559
601777   10.76

In [8]:
pd.read_csv(r'C:\Users\guqiang\Desktop\DEMA双线交叉.csv').sort_values("total_assets_cur",ascending=False)

,available_cash,total_assets_cur,total_assets_buy,buy_cur_diff,trade_times,init_cash,t1,t2
67,54625.60,179605.37,275250.37,-95645.00,19582,50000,25,100
78,45164.50,163966.35,271457.55,-107491.20,18363,50000,29,87
72,27774.75,163023.74,271445.35,-108421.61,21689,50000,27,81
59,71444.63,145160.98,228486.12,-83325.14,16716,50000,22,110
53,49192.48,137286.16,250343.76,-113057.60,18649,50000,20,100
81,19400.82,122054.23,226472.68,-104418.45,17610,50000,30,90
64,14092.43,112416.52,190237.27,-77820.75,14491,50000,24,96
35,60580.95,108807.30,155714.77,-46907.47,15309,50000,14,70
74,40778.51,103553.14,150916.72,-47363.58,12042,50000,27,135
34,17361.15,97725.62,279293.93,-181568.31,22335,50000,14,56
